In [6]:
# Parameters
y_encoding = "numeric"
y_scaling = "None"
x_encoding = [
    {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
    {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
    {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
    {"name": "prev_common_good", "norm": 128, "etype": "float"},
    {"name": "prev_valid", "etype": "bool"},
]
n_contributions = 21
n_punishments = 31
n_cross_val = 2
fraction_training = 1.0
data = "../../data/experiments/pilot_random1_player_round_slim.csv"
output_path = "../../data/training/dev"
labels = {}
model_args = {"n_layers": 2, "hidden_size": 40}
optimizer_args = {"lr": 0.001, "weight_decay": 1e-05}
train_args = {"epochs": 1000, "batch_size": 40, "clamp_grad": 1, "eval_period": 10}
device = "cpu"


In [7]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import torch as th
from aimanager.generic.data import create_syn_data, create_torch_data, get_cross_validations
from aimanager.artificial_humans.artificial_humans import ArtificialHuman
from aimanager.artificial_humans.evaluation import Evaluator

output_path = os.path.join(output_path, 'data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
df = pd.read_csv(data)
# df.head()

In [9]:
df = pd.read_csv(data)


data = create_torch_data(df)
syn_data = create_syn_data(n_contribution=21, n_punishment=31)

In [10]:
th_device = th.device(device)

metrics = []
confusion_matrix = []
syn_pred = []
ev = Evaluator()

for i, (train_data, test_data) in enumerate(get_cross_validations(data, n_cross_val)):
    model = ArtificialHuman(
        y_encoding=y_encoding, n_contributions=n_contributions, n_punishments=n_punishments, x_encoding=x_encoding,
        y_scaling=y_scaling,
        **model_args).to(th_device)

    train_data = {
        **model.encode_x(**train_data),
        **model.encode_y(**train_data),
        **train_data
    }
    train_data = {
        k: v.to(device)
        for k, v in train_data.items()
    }

    test_data = {
        **model.encode_x(**test_data),
        **model.encode_y(**test_data),
        **test_data
    }
    test_data = {
        k: v.to(device)
        for k, v in test_data.items()
    }

    syn_data_ = {
        **model.encode_x(**syn_data),
    }
    syn_data_ = {
        k: v.to(device)
        for k, v in syn_data_.items()
    }
    ev.set_data(test=test_data, train=train_data, syn=syn_data_)

    loss_fn = model.get_lossfn()

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    sum_loss = 0
    n_steps = 0
    batch_size = train_args['batch_size']

    for e in range(train_args['epochs']):
        ev.set_labels(cv_split=i, epoch=e)
        model.train()
        perm = th.randperm(train_data['ah_y_enc'].size(0))
        for start_idx in range(0, train_data['ah_y_enc'].shape[0], batch_size):
            end_idx = start_idx+batch_size
            idx = perm[start_idx:end_idx]
            batch_data = {
                 k: v[idx]
                for k, v in train_data.items()
            }

            optimizer.zero_grad()

            py = model(**batch_data).flatten(end_dim=-2)
            y_true = batch_data['ah_y_enc'].flatten(end_dim=-2)
            mask = batch_data['valid'].flatten()

            loss = loss_fn(py, y_true)

            loss = (loss * mask).sum() / mask.sum()

            loss.backward()

            if train_args['clamp_grad']:
                for param in model.parameters():
                    param.grad.data.clamp_(-train_args['clamp_grad'], train_args['clamp_grad'])
            optimizer.step()
            sum_loss += loss.item()
            n_steps +=1
        
        if e % train_args['eval_period'] == 0:
            avg_loss = sum_loss/n_steps
            print(f'CV {i} | Epoch {e} | Loss {avg_loss}')
            ev.add_loss(avg_loss)
            ev.eval_set(model, 'train')
            ev.eval_set(model, 'test')
            sum_loss = 0
            n_steps = 0

    ev.eval_sync(model)

ev.save(output_path, labels)
model_path = os.path.join(output_path, 'model.pt')
model.save(model_path)

None
-0.2553790509700775 0.018584780395030975 -0.10594632476568222 0.0 1.0 0.4409765601158142
-0.2430017739534378 0.02258904278278351 -0.09387584030628204 0.0 1.0 0.43042969703674316
-0.22162914276123047 0.03398381173610687 -0.08158253878355026 0.0 1.0 0.4866601526737213
-0.1801900565624237 0.023420587182044983 -0.07264979928731918 0.0 1.0 0.4580208361148834
CV 0 | Epoch 0 | Loss 0.46838928014039993
-0.1869477480649948 0.055820465087890625 -0.056331753730773926 0.0 1.0 0.47654294967651367
-0.18168684840202332 0.05049024522304535 -0.0451933816075325 0.0 1.0 0.43708983063697815
-0.15744149684906006 0.07618683576583862 -0.03246837854385376 0.0 1.0 0.45533204078674316
-0.14052434265613556 0.09102551639080048 -0.02141813188791275 0.0 1.0 0.42895832657814026
-0.13144376873970032 0.10513785481452942 -0.007000158540904522 0.0 1.0 0.4528515934944153
-0.10940596461296082 0.11569568514823914 0.005258943419903517 0.0 1.0 0.47765621542930603
-0.10826915502548218 0.1294214278459549 0.016270037740468